In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import json
import data_parser
from pathlib import Path 

In [2]:
URL = "https://www.macrotrends.net/stocks/research"
page = requests.get(URL)

In [3]:
soup = BeautifulSoup(page.content, "html.parser")

In [4]:
results = soup.find_all("a", href= lambda text: "/stocks/sector/" in text.lower())
sector_dict = dict()

for result in results:
    sector_dict[result.text] = ("".join(("https://www.macrotrends.net", result["href"]))).strip()

In [5]:
for sector in sector_dict:
    
    """
    First we get the sectors
    """
    
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",sector)
    
    URL_sector = sector_dict[sector]
    page_sector = requests.get(URL_sector)
    soup_sector = BeautifulSoup(page_sector.content, "html.parser")
    pattern_sector = re.compile(r"var data = \[(.*?)\]", re.MULTILINE | re.DOTALL)
    scriptParse_sector = re.search(pattern_sector, str(soup_sector))
    
    block_sector = scriptParse_sector.groups()[0]
    res_sector = json.loads("".join(["[", block_sector, "]"]))
    
    dict_industry = dict()

    # Create the dict with the industries for each sector 
    # Get the industry asociated with its url
    for industry in res_sector:

        link = BeautifulSoup(str(industry['link']), "html.parser")
        link = link.find("a")
        link = str(link['href'])
        dict_industry[industry['zacks_x_ind_desc']] = link
        
        
    # Now we get inside the links of each industry
    # From the dict created
    for industry in dict_industry:
        print(">>>>>>>>>>>>>>>>>>>>",industry)

        URL_industry = dict_industry[industry]
        page_industry = requests.get(URL_industry)
        soup_industry = BeautifulSoup(page_industry.content, "html.parser")
        pattern_industry = re.compile(r"var data = \[(.*?)\]", re.MULTILINE | re.DOTALL)
        scriptParse_industry = re.search(pattern_industry, str(soup_industry))

        block_industry = scriptParse_industry.groups()[0]
        res_industry = json.loads("".join(["[", block_industry, "]"]))

        dict_company = dict()
        dict_company_full_name = dict()
        dict_company_country = dict()

        # Get the company asociated with its url
        for company in res_industry:

            link = BeautifulSoup(str(company['link']), "html.parser")
            link = link.find("a")
            link = str(link['href'])
            link = link.replace("stock-price-history", "financial-ratios?freq=A")
            dict_company[company['ticker']] = link
            dict_company_full_name[company['ticker']] = company['comp_name_2']
            dict_company_country[company['ticker']] = company['country_code']
    
        # Get the financial data from each company
        data_dict = dict()

        for key in dict_company.keys():

            try:
                # Get the data
                URL = dict_company[key]
                data = data_parser.parser(URL)
                data['ticker'] = key
                data['industry'] = industry
                data['sector'] = sector
                data['company_full_name'] = dict_company_full_name[key]
                data['country'] = dict_company_country[key]

                # Export the data
                filepath = Path(f'financial_ratios_data/{key}.csv')  
                filepath.parent.mkdir(parents=True, exist_ok=True)  
                data.to_csv(filepath)  

                print("============",key, "Loaded")
            except:
                print("============",key, "NOT LOADED")
                continue

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> Finance
>>>>>>>>>>>>>>>>>>>> Property, Casualty and Title Insurance
============ BRK.A NOT LOADED
============ BRK.B NOT LOADED
============ CB Loaded
============ PGR Loaded
============ TRV Loaded
============ ALL Loaded
============ ACGL Loaded
============ WRB Loaded
============ MKL Loaded
============ CINF Loaded
============ RE Loaded
============ AFG Loaded
============ CNA Loaded
============ FNF Loaded
============ RNR Loaded
============ KNSL Loaded
============ RLI Loaded
============ SIGI Loaded
============ CCCS Loaded
============ FAF Loaded
============ THG Loaded
============ AXS Loaded
============ WTM Loaded
============ MCY Loaded
============ NMIH Loaded
============ SAFT Loaded
============ PLMR Loaded
============ STC Loaded
============ PRA Loaded
============ ARGO Loaded
============ AMBC Loaded
============ UFCS Loaded
============ DGICB Loaded
============ DGICA Loaded
============ HCI Loaded
============ GBLI Loaded

============ EQC Loaded
============ OUT Loaded
============ PK Loaded
============ NHI Loaded
============ FCPT Loaded
============ JBGS Loaded
============ SLG Loaded
============ SHO Loaded
============ CTRE Loaded
============ PEB Loaded
============ DRH Loaded
============ RLJ Loaded
============ GTY Loaded
============ IVT Loaded
============ XHR Loaded
============ LTC Loaded
============ GEO Loaded
============ HPP Loaded
============ CXW Loaded
============ UNIT Loaded
============ ALEX Loaded
============ GNL Loaded
============ PGRE Loaded
============ DEA Loaded
============ SVC Loaded
============ RADI Loaded
============ PDM Loaded
============ ALX Loaded
============ ESRT Loaded
============ BDN Loaded
============ NTST Loaded
============ CHCT Loaded
============ PLYM Loaded
============ RTL Loaded
============ INN Loaded
============ GOOD Loaded
============ FPI Loaded
============ UHT Loaded
============ LAND Loaded
============ INDT Loaded
============ GMRE Loaded
==

============ FSV Loaded
============ CIGI Loaded
============ CWK Loaded
============ KW Loaded
============ TCN Loaded
============ BATRA Loaded
============ EXPI Loaded
============ NMRK Loaded
============ BATRK Loaded
============ PMT Loaded
============ RMR Loaded
============ BOC Loaded
============ HOUS Loaded
============ TRTX Loaded
============ FRPH Loaded
============ TRC Loaded
============ RDFN Loaded
============ LEGH Loaded
============ ARL Loaded
============ BRDG Loaded
============ RMAX Loaded
============ DOUG Loaded
============ FIP NOT LOADED
============ CIAN Loaded
============ MLP Loaded
============ STRS Loaded
============ OPAD Loaded
============ MAYS Loaded
============ OIG Loaded
============ LEJU Loaded
============ MDJH Loaded
============ GYRO Loaded
============ UK Loaded
============ DUO Loaded
============ DNK Loaded
>>>>>>>>>>>>>>>>>>>> Banks - Southeast
============ RF Loaded
============ FCNCA Loaded
============ SSB Loaded
============ PNFP Loaded

============ BUSE Loaded
============ SRCE Loaded
============ GABC Loaded
============ QCRH Loaded
============ PEBO Loaded
============ OSBC Loaded
============ THFF Loaded
============ MBWM Loaded
============ IBCP Loaded
============ MOFG Loaded
============ FMNB Loaded
============ MCBC Loaded
============ CIVB Loaded
============ FBIZ Loaded
============ MYFW Loaded
============ ATLO Loaded
============ EBMT Loaded
============ OVBC Loaded
============ UBCP Loaded
>>>>>>>>>>>>>>>>>>>> Insurance - Accident & Health
============ AFL Loaded
============ UNM Loaded
============ TRUP Loaded
============ EIG Loaded
============ AMSF Loaded
============ AAME Loaded
>>>>>>>>>>>>>>>>>>>> Finance - Savings & Loans
============ CFG Loaded
============ NYCB Loaded
============ TFSL Loaded
============ SFBS Loaded
============ PPBI Loaded
============ WSFS Loaded
============ BANR Loaded
============ NWBI Loaded
============ PFS Loaded
============ BHLB Loaded
============ OCFC Loaded
=======

============ ZNTL Loaded
============ ME Loaded
============ MORF Loaded
============ IOVA Loaded
============ RGNX Loaded
============ BHVN Loaded
============ PRTC Loaded
============ BCYC Loaded
============ PLRX Loaded
============ ARQT Loaded
============ ALLO Loaded
============ CDMO Loaded
============ VRDN Loaded
============ PRVB Loaded
============ ANAB Loaded
============ IDYA Loaded
============ GERN Loaded
============ KURA Loaded
============ BLTE Loaded
============ PHAR Loaded
============ NVAX Loaded
============ AGEN Loaded
============ ALEC Loaded
============ IGMS Loaded
============ ADMA Loaded
============ PNT Loaded
============ TNGX Loaded
============ ANIP Loaded
============ CMPX Loaded
============ AVTE Loaded
============ MRUS Loaded
============ HSKA Loaded
============ MIRM Loaded
============ ITOS Loaded
============ ALLK Loaded
============ AVXL Loaded
============ MCRB Loaded
============ IMTX Loaded
============ VKTX Loaded
============ SANA Loaded
===

...

============ RBLX Loaded
============ MGM Loaded
============ DKNG Loaded
============ WYNN Loaded
============ CHDN Loaded
============ BYD Loaded
============ LNW Loaded
============ MLCO Loaded
============ PENN Loaded
============ IGT Loaded
============ RRR Loaded
============ PLTK Loaded
============ SCPL Loaded
============ MCRI Loaded
============ SGHC Loaded
============ CRSR Loaded
============ GDEV Loaded
============ GDEN Loaded
============ RSI Loaded
============ ACEL Loaded
============ MYPS Loaded
============ DDI Loaded
============ DOYU Loaded
============ FLL Loaded
============ GEEX NOT LOADED
============ SKLZ Loaded
============ CNTY Loaded
============ AGS Loaded
============ CPHC Loaded
============ FAZE Loaded
============ CDRO Loaded
============ EGLX Loaded
============ BRAG Loaded
============ GMGI Loaded
============ SNAL Loaded
============ AGAE Loaded
============ GAME Loaded
============ NXTP Loaded
============ SLGG Loaded
============ LTRY Loaded
=====

============ MESA Loaded
============ ANTE Loaded
>>>>>>>>>>>>>>>>>>>> Transportation - Services
============ EXPD Loaded
============ ZTO Loaded
============ CHRW Loaded
============ TFII Loaded
============ PAC Loaded
============ ASR Loaded
============ HTZ Loaded
============ SNDR Loaded
============ XPO Loaded
============ GBTG Loaded
============ OMAB Loaded
============ HUBG Loaded
============ MATX Loaded
============ RXO NOT LOADED
============ ULH Loaded
============ CYRX Loaded
============ REVG Loaded
============ LITT Loaded
============ DESP Loaded
============ DSKE Loaded
============ GPP Loaded
============ YELL Loaded
============ USX Loaded
============ DLNG Loaded
============ PWFL Loaded
============ BEST Loaded
============ YGMZ Loaded
============ HYRE Loaded
============ FRGT Loaded
>>>>>>>>>>>>>>>>>>>> Transportation - Truck
============ ODFL Loaded
============ JBHT Loaded
============ KNX Loaded
============ LSTR Loaded
============ SAIA Loaded
============ FW

============ PTRA Loaded
============ LEV Loaded
============ GOEV Loaded
============ BLBD Loaded
============ VRM Loaded
============ MULN Loaded
============ XOS Loaded
============ VLTA Loaded
============ BRDS Loaded
============ EVTV Loaded
============ ZEV Loaded
============ PEV Loaded
============ VLCN Loaded
============ AYRO Loaded
>>>>>>>>>>>>>>>>>>>> Auto Manufacturers - Foreign
============ STLA Loaded
============ HMC Loaded
============ CNHI Loaded
============ LI Loaded
============ NIO Loaded
============ TTM Loaded
============ PSNY Loaded
============ XPEV Loaded
============ CZOO Loaded
============ SEV Loaded
============ CENN Loaded
============ SOLO Loaded
============ GP Loaded
============ VEV Loaded
============ JZXN Loaded
============ TM Loaded
>>>>>>>>>>>>>>>>>>>> Auto & Truck Original Equipment Manufacturers
============ RACE Loaded
============ MGA Loaded
============ BWA Loaded
============ LEA Loaded
============ ALV Loaded
============ GNTX Loaded
===

============ INAQ Loaded
============ GPAC Loaded
============ AEAC Loaded
============ GIA Loaded
============ CFFE Loaded
============ THCP Loaded
============ BIOS NOT LOADED
============ TOAC NOT LOADED
============ MCAA NOT LOADED
============ CITE NOT LOADED
============ PLAO NOT LOADED
============ DTRT Loaded
============ SVFB Loaded
============ MEKA Loaded
============ BTWN Loaded
============ GAQ Loaded
============ USCT Loaded
============ FRON Loaded
============ CORS Loaded
============ DSAQ Loaded
============ TGR Loaded
============ WNNR Loaded
============ TLGY NOT LOADED
============ CBRG Loaded
============ SCUA NOT LOADED
============ KAII Loaded
============ SVNA NOT LOADED
============ RENE NOT LOADED
============ DPCS NOT LOADED
============ BPAC Loaded
============ XFIN NOT LOADED
============ UTAA NOT LOADED
============ CXAC NOT LOADED
============ IRRX NOT LOADED
============ WWAC NOT LOADED
============ KNSW Loaded
============ FIAC NOT LOADED
============ A

AttributeError: 'NoneType' object has no attribute 'groups'

In [ ]:
print("FINISHED")